In [ ]:
import pandas as pd
from IPython.display import Markdown

import tb_incubator.constants as const
from tb_incubator.constants import set_project_base_path
from tb_incubator.model import build_model
from tb_incubator.plotting import set_plot_label, display_plot, plot_model_vs_actual
from tb_incubator.input import load_targets, load_param_info, get_param_table, load_genexpert_util, load_report_section

pd.options.plotting.backend = "plotly"  
project_paths = set_project_base_path("../tb_incubator/")

## Introduction

In [ ]:
#load_report_section('reporting.yaml', 'introduction')
#load_report_section('reporting.yaml', 'introduction2')

In [ ]:
targets = load_targets()
indicator_names = const.indicator_names
compartments = const.compartments

In [ ]:
#| label: fig-notif
#| fig-cap: "Case notifications for tuberculosis in Indonesia"
#| fig-pos: '!tbph'

notif = targets["notif"]
fig = notif.plot()
fig.update_layout(showlegend=False, xaxis_title='Time')#
set_plot_label(fig, indicator_names, "TB case notification")
fig.update_layout(margin={'t':10,'l':10,'b':10,'r':10})
fig.show(renderer="svg", width=500, height=300)

## Model construction

In [ ]:
param_info = load_param_info()
params = param_info["value"]

In [ ]:
model, desc = build_model(params, improved_detection=True, xpert_sensitivity=True, covid_effects=True)

#Markdown(desc)

In [ ]:
opt_params = {
    'contact_rate': 51.55970620507942,
    'self_recovery_rate': 0.3408233679174109,
    'screening_scaleup_shape': 0.055,
    'screening_inflection_time': 2011.35,
    'time_to_screening_end_asymp': 1.05,
    'progression_multiplier': 0.6657917212961489,
    'detection_reduction': 0.8304634502563091,
    'follow_up_improvement': 2.1043182131703015,
}

In [ ]:
model.run(params | opt_params)

## Results

In [ ]:
outs = model.get_derived_outputs_df()

In [ ]:
#load_report_section('reporting.yaml', 'result-compartment')

In [ ]:
file_prefix = "id"

In [ ]:
#| layout-ncol: 2
#| label: fig-compsize
#| fig-cap: "Base model of TB dynamics in Indonesia."
#| fig-subcap: 
#|   - "Compartment size"
#|   - "Percentage of latent TB infection"
#| fig-pos: '!h'

fig = outs[[f"comp_size_{c}" for c in compartments]].plot.area()
#fig.add_scatter(x=targets["census_pop"].index, y=targets["census_pop"], mode="markers", name="Census data")
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))
set_plot_label(fig, indicator_names, "Compartment size")
fig
#display_plot(fig, f"{file_prefix}_comp_size", "svg")
#fig.update_xaxes(range=[2000, 2024])

In [ ]:
load_report_section('reporting.yaml', 'result-notifprev')

In [ ]:
#| label: fig-notifprev
#| fig-cap: "Comparison between model outputs and local data for calibration targets. The red dots represent the actual data from all periods, while the green dots indicate the data specifically used for calibration targets. For prevalence, all actual data is used as the calibration target."
#| fig-subcap: 
#|   - "TB notifications"
#|   - "TB prevalence"
#| layout-ncol: 2
#| fig-pos: '!ht'

fig = plot_model_vs_actual(outs, 
                     targets["notif"],
                     "notification",
                     "Notification",
                     "",
                     "Actual data (All periods)")
fig.update_layout(showlegend=False)
fig.update_xaxes(range=[2000, 2024])
#display_plot(fig, f"{file_prefix}_notif", "svg")

In [ ]:
fig = plot_model_vs_actual(outs, 
                     targets["prevalence"],
                     "prevalence",
                     "Prevalence",
                     "",
                     "Actual data")
fig.update_layout(showlegend=False)
fig.update_xaxes(range=[2000, 2020])
#display_plot(fig, f"{file_prefix}_prevalence", "svg")

In [ ]:
fig = plot_model_vs_actual(outs, 
                     targets["incidence"],
                     "incidence",
                     "Incidence",
                     "",
                     "Actual data")
fig.update_layout(showlegend=False)
fig.update_xaxes(range=[2000, 2023])
#display_plot(fig, f"{file_prefix}_incidence", "svg")

In [ ]:
fig = outs["percentage_latent"].plot()
fig.update_layout(showlegend=False)
fig.update_xaxes(range=[2000, 2020], title_text="Year")

set_plot_label(fig, indicator_names, "Percentage of latent TB infection (%)")
fig
#display_plot(fig, f"{file_prefix}_ltbi", "svg")

In [ ]:
#load_report_section('reporting.yaml', 'result-conclusion')

\newpage
## Supplementary

In [ ]:
#| label: tbl-params
#| tbl-cap: Model parameters
#| tbl-cap-location: top

fixed_param_table = get_param_table(param_info)
#Markdown(fixed_param_table.to_markdown())


\newpage
## References